In [1]:
import pandas as pd
import numpy as np
import json
import time
from transformers import AutoModel, AutoTokenizer
from bertopic import BERTopic

C:\Users\joshu\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\joshu\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\joshu\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be 

In [2]:
model_name = "allenai/scibert_scivocab_uncased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Load the data file by reading from the JSON into a generic data object
print("Importing data and building DataFrame:")
start_time = time.time()
with open('data/combinedData.json', 'r') as f:
    data = json.load(f)
end_time = time.time()
load_time = end_time - start_time
print("Data load time", load_time)

# Convert the data to a Pandas DataFrame for additional processing
print("Data loaded, converting to DataFrame")
start_time = time.time()
df = pd.DataFrame(data)
end_time = time.time()
convert_time = end_time - start_time
print("Data successfully converted to DataFrame. Conversion time:", convert_time)

# Confirm loading and conversion
print("Data loaded, generating head summary:")
print(df.head())
column_names = df.columns.to_list()
print(column_names)

# Begin topic modeling by creating the model
descriptions = df["description"].to_list()


Importing data and building DataFrame:
Data load time 0.014996767044067383
Data loaded, converting to DataFrame
Data successfully converted to DataFrame. Conversion time: 0.003007173538208008
Data loaded, generating head summary:
          id                                              title   
0  EJ1292238  What Makes a Good-Quality Language MOOC? An Em...  \
1  EJ1333411  Predicting CEFR Levels in Learners of English:...   
2  EJ1333480  The Usefulness of Graphic Illustrations in Onl...   
3  EJ1276550  Effects of Using Machine Translation to Mediat...   
4  EJ1276510  Effect of SCMC on Foreign Language Anxiety and...   

                                              author   
0                              Luo, Rong, Ye, Zixuan  \
1  Gaillat, Thomas, Simpkin, Andrew, Ballier, Nic...   
2                                   Dziemianko, Anna   
3                Lee, Sangmin-Michelle, Briggs, Neil   
4  York, James, Shibata, Koichi, Tokutake, Hayato...   

                              

In [7]:
# Tokenize the text descriptions
tokenized_descriptions = [tokenizer.tokenize(desc) for desc in descriptions]

# Convert tokens into input features and obtain embeddings
input_ids = [tokenizer.encode(" ".join(tokens), add_special_tokens=True, truncation=True, max_length=512, padding='max_length', return_tensors='pt') for tokens in tokenized_descriptions]
embeddings = [model(ids).last_hidden_state.squeeze(0) for ids in input_ids]


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 12582912 bytes.

In [ ]:
# Initialize BERTopic with the embeddings
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(embeddings)

# Get list of topics
model.get_topic_freq().head(20)

